In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import time 
import json
import os
from os import path


url = "https://www.berliner-feuerwehr.de/"

fname = "einsatzzahlen.json"
items=[]
database=[]
timestamp_old ="00:00:00"
# Funktion Seite laden und warten bis Einsatzzaheln geladen sind #

def load_data(url):

    try:
        options = Options()
        options.headless = True
        # Declare variables and setup services
        driverService = Service('driver/chromedriver')
        driver = webdriver.Chrome(service=driverService, options=options)
        driver.get(url)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, "einsatzahlen"))
        )
        soup = BeautifulSoup(driver.page_source,'html.parser')
        
    finally:
        driver.close()
    return soup

def clean_data(next_step):
    da = next_step.find(id='einsatzahlen')
    cat = da.find_all('span','cat')[:3]
    val = da.find_all('span','val')[:3]
    timestamp = da.find_all('span','cat')[3:]

    def timestamp_clean(timestamp):
        timestamp = (timestamp[0].string)
        time_stamp = re.findall('[0-9].*',timestamp)
        return time_stamp[0]

    timestamp = timestamp_clean(timestamp)
    items = []
    for i in range(len(cat)):
        bereich_1 = (cat[i].string)
        bereich_2 = (val[i].string)
        eintrag = {bereich_1:bereich_2}
        items.append(eintrag)
    daten = str({"zeitstempel": timestamp,
                "werte":items})
   
    return daten

def neuer_eintrag(daten):
    global timestamp_old
    #print(daten[17:33])
    #print(timestamp_old)
    time_new = daten[17:33] #!!!!!  Zeitstempel aus String auslesen  !!!
    if time_new != timestamp_old:
        a = []
        if not os.path.isfile(fname):
            a.append(daten)
            with open(fname, mode='w', encoding="utf-8") as f:
                f.write(json.dumps(a,indent=2))
                
        else:
            with open(fname) as feedsjson:
                feeds = json.load(feedsjson)
                feeds.append(daten)
            with open(fname, mode='w',encoding="utf-8") as f:
                f.write(json.dumps(feeds, indent=2))
                
        return
    else:
        return None

def main():

    while True:
        next_step =load_data(url)
        #print(next_step)
        daten = clean_data(next_step)
        neuer_eintrag(daten)
        #print(daten)
       
        
        
        time.sleep(5)
        

In [2]:
main()

UnboundLocalError: local variable 'driver' referenced before assignment